In [33]:
import uuid
from typing import TypedDict, NotRequired
from langchain.tools import tool
from langchain.agents import create_agent
from langchain.agents.middleware import ModelRequest, ModelResponse, AgentMiddleware
from langchain.messages import SystemMessage
from langgraph.checkpoint.memory import InMemorySaver
from typing import Callable

# Define skill structure
class Skill(TypedDict):
    """A skill that can be progressively disclosed to the agent."""
    name: str
    description: str
    content: str

# Define skills with schemas and business logic
SKILLS: list[Skill] = [
    {
        "name": "financial_health_indicators",
        "description": "Business logic around the financial health indicators",
        "content": """

# Financial Health Context
- Indicators: Spending, Reserve, Insurance, Investment, Pension
- Progression phases of financial health
    1. Get spending under control
    2. Build reserve & cover basic insurances
    3. Invest surplus & prepare for pension

## Status meanings
- Statuses where customer is considered financially vulnerable
    - UNHEALTHY: Immediate attention needed
    - CAUTION: Close to risk, improve gradually
    - REVIEW: (valid only for Insurance) basic insurance missing, that risk needs to be covered next
- Statuses where customer is considered financially healthy
    - FIT: Healthy, move to next priority
    - POTENTIAL: Surplus available, redirect to next indicator/phase
- Other statuses
    - USER_INPUT_NEEDED: Ask user to provide missing data

# Task
    Generate advice for the given customer aiming to improve the financial health indicators, following the Guidelines and the JSON schema structure

# Indicator descriptions
## Spending indicator
### Goal
    Customers cover regular costs from regular income, spend wisely, and meet all regular financial obligations
### Platform functionality description
    - The platform calculates the income and costs of the customer based on the transaction on their banking accounts
    - If the costs or income can not be calculated due to missing data, then the indicator is returned as USER_INPUT_NEEDED and the user can add the missing data manually

## Reserve indicator
### Goal
    Customers build a sufficient financial reserve in liquid assets to be prepared for unexpected financial shocks. Customers can cover regular expenses in case of income interruption. Should be in focus after the spending is under control.
### Platform functionality description
    - The platform functionality determines the optimal reserve range based on the customer's income and expenses. The customer's reserve is compared to the optimal range.
    - The customer can also add a reserve manually, in case they have a reserve in another bank account, or cash available.

## Insurance indicator
### Goal
    Customers limit relevant financial risks with suitable insurance protection, regardless the size of the financial reserve available. The basic insurances are meant to protect customers from unexpected financial shocks.
### Platform functionality description
    - A platform functionality exists that shows detected insurances, which are paid by the customer from his own account. If risk type could not be determined, then the customer is asked to provide it.
    - The customer can also add insurances that they have with other companies (also called External insurances)

## Investment indicator
### Goal
    Customers actively manage their surplus liquidity, beyond their financial reserves, to invest in line with their financial goals and risk preferences.
### Platform functionality description
    - The customer can add investments that they have with other companies (also called External investments) and also indicate if they invest regularly or not

## Pension indicator
### Goal
    Customers actively preparing for retirement by taking concrete actions to ensure they can maintain their living standard once they retire
""",
    },
    {
        "name": "financial_advice",
        "description": "To advise on financial health, the order in which the financial health indicators should be tackled needs to be determined",
        "content": """


          # Next best indicator
          When asked about financial health adivce, write one paragraph for each of the items from the get_next_best_indicators tool
        """
    },
    {
        "name": "financial_summary",
        "description": "To summarize the financial health first need to understand financial health indicators",
        "content": """

            # Summary of the financial health
            When asked about financial health situation, structure the response in 2 paragraphs:
              i. be optimistic and praise on the good aspects of the financial situation
              ii. describe the areas that need attention and write the actions the customer should take and describe why this is important to act
        """
    },
]

# Create skill loading tool
@tool
def load_skill(skill_name: str) -> str:
    """Load the full content of a skill into the agent's context.

    Use this when you need detailed information about how to handle a specific
    type of request. This will provide you with comprehensive instructions,
    policies, and guidelines for the skill area.

    Args:
        skill_name: The name of the skill to load (e.g., "next_best_indicator", "financial_health_indicators")
    """
    # Find and return the requested skill
    for skill in SKILLS:
        if skill["name"] == skill_name:
            return f"Loaded skill: {skill_name}\n\n{skill['content']}"

    # Skill not found
    available = ", ".join(s["name"] for s in SKILLS)
    return f"Skill '{skill_name}' not found. Available skills: {available}"


@tool
def get_financial_health_indicators():
    """
    Returns 5 financial health indicators with fixed statuses.
    """
    return {
        "spending": "FIT",
        "reserve": "CAUTION",
        "insurance": "UNHEALTHY",
        "investment": "FIT",
        "pension": "FIT"
    }

@tool
def get_next_best_indicators():
    """
    Returns the order in which the financial health indicators need to be tackled.
    """
    return [ "insurance", "reserve" ]


# Create skill middleware
class SkillMiddleware(AgentMiddleware):
    """Middleware that injects skill descriptions into the system prompt."""

    # Register the load_skill tool as a class variable
    tools = [load_skill, get_financial_health_indicators, get_next_best_indicators]

    def __init__(self):
        """Initialize and generate the skills prompt from SKILLS."""
        # Build skills prompt from the SKILLS list
        skills_list = []
        for skill in SKILLS:
            skills_list.append(
                f"- **{skill['name']}**: {skill['description']}"
            )
        self.skills_prompt = "\n".join(skills_list)

    def wrap_model_call(
        self,
        request: ModelRequest,
        handler: Callable[[ModelRequest], ModelResponse],
    ) -> ModelResponse:
        """Sync: Inject skill descriptions into system prompt."""
        # Build the skills addendum
        skills_addendum = (
            f"\n\n## Available Skills\n\n{self.skills_prompt}\n\n"
            "Use the load_skill tool when you need detailed information "
            "about handling a specific type of request."
        )

        # Append to system message content blocks
        new_content = list(request.system_message.content_blocks) + [
            {"type": "text", "text": skills_addendum}
        ]
        new_system_message = SystemMessage(content=new_content)
        modified_request = request.override(system_message=new_system_message)
        return handler(modified_request)

In [10]:
!pip install langchain-mistralai

In [35]:
# Initialize your chat model (replace with your model)
# Example: from langchain_anthropic import ChatAnthropic
# model = ChatAnthropic(model="claude-3-5-sonnet-20241022")

import os
# Set your Mistral API key here or ensure it's set as an environment variable
# IMPORTANT: Replace 'YOUR_VALID_MISTRAL_API_KEY' with your actual Mistral API key.
# For better security, consider using Colab's 'Secrets' feature to store your API key.
from google.colab import userdata
API_KEY = userdata.get('Mistral')

from langchain_mistralai import ChatMistralAI
model = ChatMistralAI(model="mistral-small", api_key=API_KEY)

# Create the agent with skill support
agent = create_agent(
    model,
    system_prompt=(
        "You are a banking assistant that helps users "
        "improve their financial health"
    ),
    middleware=[SkillMiddleware()],
    checkpointer=InMemorySaver(),
)

# Example usage
if __name__ == "__main__":
    # Configuration for this conversation thread
    thread_id = str(uuid.uuid4())
    config = {"configurable": {"thread_id": thread_id}}

    # Ask for a SQL query
    result = agent.invoke(
        {
            "messages": [
                {
                    "role": "user",
                    "content": (
                        "How does my financial health look like?"
                    ),
                }
            ]
        },
        config
    )

    # Print the conversation
    for message in result["messages"]:
        if hasattr(message, 'pretty_print'):
            message.pretty_print()
        else:
            print(f"{message.type}: {message.content}")

================================ Human Message =================================

How does my financial health look like?
================================== Ai Message ==================================
Tool Calls:
  load_skill (0pBi32nVr)
 Call ID: 0pBi32nVr
  Args:
    skill_name: financial_summary
================================= Tool Message =================================
Name: load_skill

Loaded skill: financial_summary


      
            # Summary of the financial health
            When asked about financial health situation, structure the response in 2 paragraphs:
              i. be optimistic and praise on the good aspects of the financial situation
              ii. describe the areas that need attention and write the actions the customer should take and describe why this is important to act
        
================================== Ai Message ==================================
Tool Calls:
  get_financial_health_indicators (YszHSse5H)
 Call ID: YszHSse5H
  Args:
====